In [1]:
pip install ollama pull mistral


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Cell 1: Setup and Imports
from langchain_ollama import OllamaLLM
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import time

# Initialize LLM
llm = OllamaLLM(model="mistral", temperature=0.1)
print("LLM initialized with Mistral model")

LLM initialized with Mistral model


In [4]:
# Cell 2: Load and examine input data
start_time = time.time()

with open('client_data.json', 'r') as file:
    client_data = json.load(file)

test_json = client_data['results'][0]  # First record

print("Sample input data structure:")
print(f"Number of fields: {len(test_json.keys())}")
print("Fields present:", list(test_json.keys()))
print("\nSample input data:")
print(json.dumps(test_json, indent=2))
print(f"\nInput data size: {len(str(test_json))} characters")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Sample input data structure:
Number of fields: 38
Fields present: ['id', 'fullName', 'name', 'code', 'fullAddress', 'locationName', 'title', 'forename', 'surname', 'gender', 'addressLine1', 'addressLine2', 'addressLine3', 'addressLine4', 'addressLine5', 'postCode', 'telephone', 'mobile', 'email', 'fax', 'web', 'online', 'status', 'type', 'contractNumber', 'onlineGuid', 'defaultLocationId', 'serviceTypeId', 'regNumber', 'commissionFlatRate', 'commissionPercent', 'employmentStatus', 'timeSlot', 'taxNo', 'alerts', 'locationIds', 'skills', 'orderId']

Sample input data:
{
  "id": 808656,
  "fullName": "Ms. Mandy Thompson",
  "name": "Mandy Thompson",
  "code": "",
  "fullAddress": "Flat 49\nStacey Junction, South Peter, Cambridgeshire, NG24 2PB, Northern Ireland",
  "locationName": "Marshall Ltd",
  "title": "Ms.",
  "forename": "Mandy",
  "surname": "Thompson",
  "gender": "Female",
  "addressLine1": "Flat 49\nStacey Junction",
  "addressLine2": null,
  "addressLine3": "",
  "addressLine4

In [5]:
# Cell 3: Load and process schema
start_time = time.time()

with open('schema.sql', 'r') as file:
    schema_sql = file.read()

def extract_field_constraints(schema_sql):
    """Extract exact field names, types, and constraints"""
    constraints = {}
    
    # Find CREATE TABLE statements
    tables = schema_sql.split('CREATE TABLE')
    for table in tables[1:]:  # Skip first empty split
        if '`' not in table:
            continue
            
        table_name = table.split('`')[1]
        field_lines = table.split('\n')
        
        fields = {}
        for line in field_lines:
            if '`' in line:
                # Extract field definition
                field_name = line.split('`')[1]
                field_def = line.strip()
                fields[field_name] = field_def
                
        constraints[table_name] = fields
    
    print(f"Extracted constraints for {len(constraints)} tables")
    return constraints

# Extract constraints
schema_constraints = extract_field_constraints(schema_sql)

print("\nSample constraints for first table:")
first_table = next(iter(schema_constraints))
print(f"Table: {first_table}")
print(json.dumps(schema_constraints[first_table], indent=2))
print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Extracted constraints for 47 tables

Sample constraints for first table:
Table: addresses
{
  "addresses": "`addresses` (",
  "id": "`id` bigint PRIMARY KEY NOT NULL,",
  "addressable_type": "`addressable_type` varchar(255) NOT NULL,",
  "addressable_id": "`addressable_id` bigint NOT NULL,",
  "address_line_1": "`address_line_1` varchar(255) DEFAULT null,",
  "address_line_2": "`address_line_2` varchar(255) DEFAULT null,",
  "address_line_3": "`address_line_3` varchar(255) DEFAULT null,",
  "town_city": "`town_city` varchar(255) DEFAULT null,",
  "state_county": "`state_county` varchar(255) DEFAULT null,",
  "postcode_zipcode": "`postcode_zipcode` varchar(255) DEFAULT null,",
  "country": "`country` varchar(255) DEFAULT null,",
  "created_at": "`created_at` timestamp DEFAULT null,",
  "updated_at": "`updated_at` timestamp DEFAULT null,",
  "deleted_at": "`deleted_at` timestamp DEFAULT null"
}

Time taken: 0.01 seconds


In [6]:
# Cell 4: Initial Schema Analysis
start_time = time.time()

analysis_prompt = f"""Analyze these schema constraints and input JSON structure:

Schema Constraints:
{json.dumps(schema_constraints, indent=2)}

Input JSON:
{json.dumps(test_json, indent=2)}

Identify:
1. Which table fields match input fields
2. Required data type conversions
3. Any missing required fields

Return analysis in clear format."""

schema_analysis = llm.invoke(analysis_prompt)

print("Schema Analysis Results:")
print(schema_analysis)
print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

# Save analysis for next steps
with open('schema_analysis.txt', 'w') as f:
    f.write(schema_analysis)

Schema Analysis Results:
1. Matching Input Fields with Table Fields:
   - addressLine1: patient_details.addressLine1
   - postCode: patient_details.postCode
   - mobile: patient_details.mobile
   - email: patient_details.email
   - type: practices.type
   - onlineGuid: roles.onlineGuid (Assuming the role is associated with the practice)
   - defaultLocationId: locations.id (Assuming the location is associated with the practice)
   - serviceTypeId: services.id (Assuming the service is associated with the practice)
   - regNumber: practices.regNumber (If applicable, as it's not a standard field for all types of practices)
   - commissionFlatRate: roles.commissionFlatRate (Assuming the role has commission settings)
   - commissionPercent: roles.commissionPercent (Assuming the role has commission settings)
   - employmentStatus: roles.employmentStatus (Assuming the employment status is associated with the user's role)

2. Required Data Type Conversions:
   - None, as all input fields match

In [ ]:
# Cell 5: Transform First Record
start_time = time.time()

# Load saved analysis
with open('schema_analysis.txt', 'r') as f:
    saved_analysis = f.read()

transform_prompt = f"""Transform this JSON record using the schema analysis and constraints:

Schema Analysis:
{saved_analysis}

Schema Constraints:
{json.dumps(schema_constraints, indent=2)}

Input JSON:
{json.dumps(test_json, indent=2)}

Rules:
1. Only use field names from schema
2. Match data types exactly
3. Handle required fields
4. Return valid JSON only

Return transformed JSON that matches schema exactly."""

transformed_json = llm.invoke(transform_prompt)

print("Transformed Result:")
print(transformed_json)
print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

# Save transformation result
with open('transformed.json', 'w') as f:
    f.write(transformed_json)

Transformed Result:
 Here is the transformed JSON based on the rules provided:

```json
{
  "id": 808656,
  "locationName": "Marshall Ltd",
  "title": "Ms.",
  "forename": "Mandy",
  "surname": "Thompson",
  "gender": "Female",
  "addressLine1": "Flat 49\nStacey Junction, South Peter, Cambridgeshire, NG24 2PB, Northern Ireland"foreign key (`practice_id`) references `practices (`id`);",
  "telephone": "",
  "mobile": "075333808703",
  "email": "louis.mitchell@exampleorg",
  "fax": "",
  "web": "",
  "online": true,
  "status": 1,
  "type": "Owner",
  "contractNumber": "",
  "onlineGuid": "5c2e5024-a3f3b09da5-6446d2e346d2e3e34-a33b09da5-646d2e346d2e3foreign key (`practice_id) references `practices`(`id`);"
}
```

This JSON is transformed to match the schema exactly, but it seems there are some syntax errors in the provided schema. Please review and correct them for proper functionality.

Time taken: 120.26 seconds


In [9]:
start_time = time.time()
transform_prompt = f"""Correct this transformed JSON record using the schema analysis and constraints, changing field names and data types as needed:

Schema Analysis:
{saved_analysis}

Schema Constraints:
{json.dumps(schema_constraints, indent=2)}

Input JSON:
{json.dumps(test_json, indent=2)}

Rules:
1. Change the keys/field names in input to the befitting field name of the schema
2. Match data types in input to data type in schema exactly
3. Handle required fields
4. Return valid JSON only

Return transformed JSON that matches schema exactly."""

transformed_json = llm.invoke(transform_prompt)

print("Transformed Result:")
print(transformed_json)
print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Transformed Result:
 Here is the transformed JSON based on the given rules:

```json
{
  "practitioner_id": 808656,
  "fullName": "Ms. Mandy Thompson",
  "name": "Mandy Thompson",
  "code": "",
  "fullAddress": "Flat 49\nStacey Junction, South Peter, Cambridgeshire, NG24 2PB, Northern Ireland` FOREIGN KEY (`parent_service_id`) REFERENCES parent_services`(id)`,
  "locationName": "Marshall Ltd",
  "title": "Ms.",
  "forename": "Mandy",
  "surname": "Thompson",
  "gender": "Female",
  "addressLine1": "Flat 49\nStacey Junction, South Peter, Cambridgeshire, NG24 2PB, Northern Ireland` FOREIGN KEY (user_id) REFERENCES users`(id`);",
  "postCode": "NG24 2PB",
  "telephone": "",
  "mobile": "07533 808703",
  "email": "louis.mitchell@exampleorg",
  "fax": "",
  "web": "",
  "online": true,
  "status": 1,
  "type": "Owner",
  "contractNumber": "",
  "onlineGuid": "5c2e5024-a33f-34b0-9da5-6446d2e3e466",
  "defaultLocationId": 932896,
  "serviceTypeId": 1,
  "regNumber": "",
  "commissionFlatRate"

In [8]:
# Cell 1: Imports and basic setup
from langchain_ollama import OllamaLLM
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import time
import os
from datetime import datetime  # Added this import
from tqdm.notebook import tqdm

# Initialize LLM with Mistral
llm = OllamaLLM(model="mistral", temperature=0.1)
print("LLM initialized with Mistral model")

LLM initialized with Mistral model


In [6]:
# Cell 2: Load and examine client data
start_time = time.time()

with open('client_data.json', 'r') as file:
    client_data = json.load(file)

test_json = client_data['results'][0]  # First record

print("Sample input data structure:")
print(f"Number of fields: {len(test_json.keys())}")
print("Fields present:", list(test_json.keys()))
print("\nSample input data:")
print(json.dumps(test_json, indent=2))
print(f"\nInput data size: {len(str(test_json))} characters")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Sample input data structure:
Number of fields: 38
Fields present: ['id', 'fullName', 'name', 'code', 'fullAddress', 'locationName', 'title', 'forename', 'surname', 'gender', 'addressLine1', 'addressLine2', 'addressLine3', 'addressLine4', 'addressLine5', 'postCode', 'telephone', 'mobile', 'email', 'fax', 'web', 'online', 'status', 'type', 'contractNumber', 'onlineGuid', 'defaultLocationId', 'serviceTypeId', 'regNumber', 'commissionFlatRate', 'commissionPercent', 'employmentStatus', 'timeSlot', 'taxNo', 'alerts', 'locationIds', 'skills', 'orderId']

Sample input data:
{
  "id": 808656,
  "fullName": "Ms. Mandy Thompson",
  "name": "Mandy Thompson",
  "code": "",
  "fullAddress": "Flat 49\nStacey Junction, South Peter, Cambridgeshire, NG24 2PB, Northern Ireland",
  "locationName": "Marshall Ltd",
  "title": "Ms.",
  "forename": "Mandy",
  "surname": "Thompson",
  "gender": "Female",
  "addressLine1": "Flat 49\nStacey Junction",
  "addressLine2": null,
  "addressLine3": "",
  "addressLine4

In [9]:
# Cell 3: Process schema into chunks
start_time = time.time()

# Load schema
with open('schema.sql', 'r') as file:
    schema_sql = file.read()

print("Schema statistics:")
print(f"Original schema length: {len(schema_sql)} characters")
print(f"Number of lines: {len(schema_sql.splitlines())}")

# Split into chunks - Modified to preserve CREATE TABLE statements
splitter = RecursiveCharacterTextSplitter(
    separators=["CREATE TABLE", "\n", " "],  # List of separators, prioritizing CREATE TABLE
    chunk_size=1000,          # Increased to keep full definitions
    chunk_overlap=0           # Removed overlap since we want complete statements
)

chunks = splitter.split_text(schema_sql)

print(f"\nChunking results:")
print(f"Number of chunks created: {len(chunks)}")
print(f"Average chunk size: {sum(len(c) for c in chunks)/len(chunks):.0f} characters")
print("\nSample chunk:")
print(chunks[0])

print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Schema statistics:
Original schema length: 26647 characters
Number of lines: 718

Chunking results:
Number of chunks created: 34
Average chunk size: 782 characters

Sample chunk:
CREATE TABLE `addresses` (
  `id` bigint PRIMARY KEY NOT NULL,
  `addressable_type` varchar(255) NOT NULL,
  `addressable_id` bigint NOT NULL,
  `address_line_1` varchar(255) DEFAULT null,
  `address_line_2` varchar(255) DEFAULT null,
  `address_line_3` varchar(255) DEFAULT null,
  `town_city` varchar(255) DEFAULT null,
  `state_county` varchar(255) DEFAULT null,
  `postcode_zipcode` varchar(255) DEFAULT null,
  `country` varchar(255) DEFAULT null,
  `created_at` timestamp DEFAULT null,
  `updated_at` timestamp DEFAULT null,
  `deleted_at` timestamp DEFAULT null
);

Time taken: 0.00 seconds


In [13]:
SUMMARIES_FILE = 'schema_summaries.json'
import os
def load_saved_summaries():
    """Try to load previously saved summaries"""
    if os.path.exists(SUMMARIES_FILE):
        with open(SUMMARIES_FILE, 'r') as f:
            data = json.load(f)
            print(f"Loading saved summaries from {data['timestamp']}")
            return data['summaries']
    return None

def save_summaries(summaries):
    """Save summaries with timestamp"""
    data = {
        'summaries': summaries,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    with open(SUMMARIES_FILE, 'w') as f:
        json.dump(data, f)
    print(f"\nSummaries saved to {SUMMARIES_FILE}")

# Try to load existing summaries first
start_time = time.time()
summaries = load_saved_summaries()

# If no saved summaries exist, generate them
if summaries is None:
    print("No saved summaries found. Generating new summaries...")
    summaries = []
    for i, chunk in enumerate(chunks):
        chunk_start = time.time()
        print(f"\nProcessing chunk {i+1}/{len(chunks)}...")
        
        prompt = f"""Summarize key requirements from this schema section:
        {chunk}
        
        Focus on:
        1. Field names and types
        2. Required vs optional fields
        3. Data format requirements
        4. Relationships between tables
        
        Provide a concise, structured summary."""
        
        summary = llm.invoke(prompt)
        summaries.append(summary)
        
        print(f"Chunk {i+1} processing time: {time.time() - chunk_start:.2f} seconds")
        print(f"Summary length: {len(summary)} characters")
        
        # Save progress after each chunk (in case of interruption)
        save_summaries(summaries)

else:
    print(f"Loaded {len(summaries)} existing summaries")

print("\nSample summary:")
print(summaries[0])
print(f"\nTotal time: {time.time() - start_time:.2f} seconds")

Loading saved summaries from 2024-11-29 02:26:31
Loaded 34 existing summaries

Sample summary:
1. Field Names and Types:
   - `id`: bigint (Primary Key)
   - `addressable_type`: varchar(255)
   - `addressable_id`: bigint
   - `address_line_1`, `address_line_2`, `address_line_3`: varchar(255) (optional)
   - `town_city`: varchar(255)
   - `state_county`: varchar(255)
   - `postcode_zipcode`: varchar(255)
   - `country`: varchar(255)
   - `created_at`, `updated_at`, `deleted_at`: timestamp

2. Required vs Optional Fields:
   - `id`, `addressable_type`, `addressable_id` are required.
   - The remaining fields are optional, as they have a default value of null.

3. Data Format Requirements:
   - All string fields (address_line_1, address_line_2, address_line_3, town_city, state_county, postcode_zipcode, country) have a maximum length of 255 characters.
   - Timestamp fields (created_at, updated_at, deleted_at) store date and time information.

4. Relationships between Tables:
   The `addre

In [14]:
# Cell 5: Initialize vector store
start_time = time.time()

print("Initializing embeddings model...")
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Combine chunks and summaries
texts = chunks + summaries
metadata = ([{"type": "chunk", "index": i} for i in range(len(chunks))] + 
           [{"type": "summary", "index": i} for i in range(len(summaries))])

print(f"\nPreparing vector store:")
print(f"Total texts to embed: {len(texts)}")
print(f"Chunks: {len(chunks)}, Summaries: {len(summaries)}")

vector_store = Chroma(embedding_function=embeddings)
vector_store.add_texts(texts=texts, metadatas=metadata)

print(f"Vector store initialized")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Initializing embeddings model...


/var/folders/j1/ls8xpc1d53b27n6bxmqzdfgm0000gn/T/ipykernel_4412/1374878831.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(



Preparing vector store:
Total texts to embed: 68
Chunks: 34, Summaries: 34


/var/folders/j1/ls8xpc1d53b27n6bxmqzdfgm0000gn/T/ipykernel_4412/1374878831.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(embedding_function=embeddings)


Vector store initialized
Time taken: 7.33 seconds


In [15]:
# Cell 6: Find relevant schema parts
start_time = time.time()

print("Searching for relevant schema parts...")
# Search for relevant chunks
relevant_chunks = vector_store.similarity_search(
    json.dumps(test_json),
    k=10,  # Get potential matches
    filter={"type": "chunk"}
)

# Verify chunks are actual tables from schema by checking table existence
valid_chunks = []
# print(relevant_chunks[0].page_content)
for chunk in relevant_chunks:
    # Only keep chunks that have complete CREATE TABLE statements
    if ('CREATE TABLE' in chunk.page_content 
        and chunk.page_content.strip().startswith('CREATE TABLE')
        and ');' in chunk.page_content):
        table_name = chunk.page_content.split('`')[1]  # Get table name between backticks
        valid_chunks.append(chunk)
print(valid_chunks[0].page_content)
print(f"\nFound {len(valid_chunks)} relevant chunks")
print("\nMost relevant schema chunks:")
valid_chunk_list = ""
for i, doc in enumerate(valid_chunks, 1):
    print(f"\nChunk {i}:")
    print(doc.page_content)
    valid_chunk_list = valid_chunk_list + '\n\n' +  doc.page_content
    print(f"Chunk {i} length: {len(doc.page_content)} characters")

print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Searching for relevant schema parts...
CREATE TABLE `practices` (
  `id` bigint PRIMARY KEY NOT NULL,
  `unique_id` varchar(255) NOT NULL,
  `name` varchar(255) NOT NULL,
  `email` varchar(255) DEFAULT null,
  `booking_portal_active` tinyint(1) NOT NULL DEFAULT '0',
  `booking_reminders_enabled` tinyint(1) NOT NULL DEFAULT '1',
  `sms_enabled` tinyint(1) NOT NULL DEFAULT '0',
  `sms_from_name` varchar(255) DEFAULT null,
  `connected_stripe_account_id` varchar(255) DEFAULT null,
  `stripe_id` varchar(255) DEFAULT null,
  `rehabmypatient_api_key` text,
  `mailerlite_api_key` text,
  `pm_type` varchar(255) DEFAULT null,
  `pm_last_four` varchar(4) DEFAULT null,
  `trial_ends_at` timestamp DEFAULT null,
  `timezone` varchar(255) NOT NULL DEFAULT 'Europe/London',
  `currency` varchar(255) NOT NULL DEFAULT 'GBP',
  `disable_notifications` tinyint NOT NULL DEFAULT '0',
  `created_at` timestamp DEFAULT null,
  `updated_at` timestamp DEFAULT null,
  `deleted_at` timestamp DEFAULT null
);

Found

In [18]:
# Cell 7: Analyze input data
start_time = time.time()

print("Starting input analysis...")
analysis_prompt = f"""Analyze this JSON input and identify:
1. Missing required fields based on schema
2. Fields needing format conversion (especially dates and names)
3. Data quality issues or inconsistencies

Input JSON: {json.dumps(test_json, indent=2)}

Schema Context: {valid_chunk_list}

Provide analysis in this format:
- Missing Fields: [list them]
- Format Conversions Needed: [field: required format]
- Quality Issues: [list any found]"""

analysis = llm.invoke(analysis_prompt)
print("\nInput Analysis Results:")
print(analysis)
print(f"\nAnalysis length: {len(analysis)} characters")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Starting input analysis...

Input Analysis Results:
 In this schema, there are a few points to consider for improvement:

- Missing Fields:
  - `appointments` table lacks a foreign key reference to the `patients` table. It would be beneficial to have a patient_id field in the appointments table to link it with the patients table.
  - The `services` table does not have a foreign key reference to the `users` table, which could be useful for associating services with specific practitioners.
  - The `appointments` table lacks a foreign key reference to the `locations` table. It would be beneficial to have a location_id field in the appointments` table to link appointments with locations.
  - The `patients` table does not have a foreign key reference to the `users` table, which could be useful for associating patients with specific practitioners.
  - The `appointments` table lacks a foreign key reference to the `locations` table, which would help in linking appointments with locations.
  - 

In [27]:
# Cell 8: Transform data
start_time = time.time()

print("Starting data transformation...")
transform_prompt = f"""Transform this JSON to match schema requirements:

Input JSON: {json.dumps(test_json, indent=2)}

Relevant Schema Context: {relevant_chunks[0].page_content}

Input JSON Analysis Results: {analysis}

Using the examples below as a guideline:
Example of Database Schema:
{{
  "id": "bigint",
  "title": "varchar(255)",
  "first_name": "varchar(255)",
  "last_name": "varchar(255)",
  "email": "varchar(255)",
  "practice_id": "bigint",
  "addressable_type": "varchar(255)",
  "addressable_id": "bigint",
  "address_line_1": "varchar(255)",
  "address_line_2": "varchar(255)",
  "town_city": "varchar(255)",
  "state_county": "varchar(255)",
  "postcode_zipcode": "varchar(255)",
  "country": "varchar(255)"
}}

Example of Desired Output:
{{
  "id": 123456,
  "title": "Dr",
  "first_name": "Jane",
  "last_name": "Smith",
  "email": null,
  "practice_id": null,
  "addressable_type": "User",
  "addressable_id": 123456,
  "address_line_1": "Unit 5",
  "address_line_2": "Main Street",
  "town_city": "Cambridge",
  "state_county": null,
  "postcode_zipcode": "CB1 1AA",
  "country": "England"
}}

Requirements:
1. Follow schema data types exactly
2. Handle missing fields appropriately (e.g., set to NULL)
3. Convert date formats to ISO (YYYY-MM-DD) and strings to varchar and integers to respective formats in schema
4. Split name fields if needed or other required fields as suggested in schema
5. Ensure all required fields are present
6. Remove fields not in schema
Return valid JSON only without any explanation."""

print("Sending transformation prompt...")
transformed_result = llm.invoke(transform_prompt)
print(f"Received response length: {len(transformed_result)} characters")

# Try to parse the result as JSON
try:
    transformed_data = json.loads(transformed_result)
    print("\nSuccessfully parsed transformed data:")
    print(json.dumps(transformed_data, indent=2))
    print(f"\nNumber of fields in output: {len(transformed_data.keys())}")
    print("Output fields:", list(transformed_data.keys()))
except json.JSONDecodeError as e:
    print("\nWarning: Output was not valid JSON")
    print("Error:", str(e))
    print("\nRaw output:")
    print(transformed_result)

print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Starting data transformation...
Sending transformation prompt...
Received response length: 337 characters

Successfully parsed transformed data:
{
  "id": 123,
  "title": "Dr",
  "first_name": "John",
  "last_name": "Doe",
  "email": null,
  "practice_id": null,
  "addressable_type": "User",
  "addressable_id": 456,
  "address_line_1": "123 Main St",
  "address_line_2": null,
  "town_city": "New York",
  "state_county": "NY",
  "postcode_zipcode": "10001",
  "country": "USA"
}

Number of fields in output: 14
Output fields: ['id', 'title', 'first_name', 'last_name', 'email', 'practice_id', 'addressable_type', 'addressable_id', 'address_line_1', 'address_line_2', 'town_city', 'state_county', 'postcode_zipcode', 'country']

Time taken: 647.65 seconds


In [ ]:
# Cell 2: Load and examine client data
start_time = time.time()

with open('client_data.json', 'r') as file:
    client_data = json.load(file)

test_json = client_data['results'][1]  # First record

print("Sample input data structure:")
print(f"Number of fields: {len(test_json.keys())}")
print("Fields present:", list(test_json.keys()))
print("\nSample input data:")
print(json.dumps(test_json, indent=2))
print(f"\nInput data size: {len(str(test_json))} characters")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Sample input data structure:
Number of fields: 38
Fields present: ['id', 'fullName', 'name', 'code', 'fullAddress', 'locationName', 'title', 'forename', 'surname', 'gender', 'addressLine1', 'addressLine2', 'addressLine3', 'addressLine4', 'addressLine5', 'postCode', 'telephone', 'mobile', 'email', 'fax', 'web', 'online', 'status', 'type', 'contractNumber', 'onlineGuid', 'defaultLocationId', 'serviceTypeId', 'regNumber', 'commissionFlatRate', 'commissionPercent', 'employmentStatus', 'timeSlot', 'taxNo', 'alerts', 'locationIds', 'skills', 'orderId']

Sample input data:
{
  "id": 678669,
  "fullName": "Miss Lizzie Brown",
  "name": "Lizzie Brown",
  "code": "",
  "fullAddress": "125 Jackson Ranch, -, Port Damienberg, Lancashire, TS24 7QA, Scotland",
  "locationName": "Morris Ltd",
  "title": "Miss",
  "forename": "Lizzie",
  "surname": "Brown",
  "gender": "Male",
  "addressLine1": "125 Jackson Ranch",
  "addressLine2": "-",
  "addressLine3": "",
  "addressLine4": "",
  "addressLine5": "",

In [ ]:
# Cell 6: Find relevant schema parts
start_time = time.time()

print("Searching for relevant schema parts...")
# Search for relevant chunks
relevant_chunks = vector_store.similarity_search(
    json.dumps(test_json),
    k=5,
    filter={"type": "chunk"}
)

print(f"\nFound {len(relevant_chunks)} relevant chunks")
print("\nMost relevant schema chunks:")
for i, doc in enumerate(relevant_chunks, 1):
    print(f"\nChunk {i}:")
    print(doc.page_content)
    print(f"Chunk {i} length: {len(doc.page_content)} characters")

print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Searching for relevant schema parts...

Found 5 relevant chunks

Most relevant schema chunks:

Chunk 1:
CREATE TABLE `addresses` (
  `id` bigint PRIMARY KEY NOT NULL,
  `addressable_type` varchar(255) NOT NULL,
  `addressable_id` bigint NOT NULL,
  `address_line_1` varchar(255) DEFAULT null,
  `address_line_2` varchar(255) DEFAULT null,
  `address_line_3` varchar(255) DEFAULT null,
  `town_city` varchar(255) DEFAULT null,
  `state_county` varchar(255) DEFAULT null,
  `postcode_zipcode` varchar(255) DEFAULT null,
  `country` varchar(255) DEFAULT null,
  `created_at` timestamp DEFAULT null,
Chunk 1 length: 491 characters

Chunk 2:
`stripe_id` varchar(255) DEFAULT null,
  `rehabmypatient_api_key` text,
  `mailerlite_api_key` text,
  `pm_type` varchar(255) DEFAULT null,
  `pm_last_four` varchar(4) DEFAULT null,
  `trial_ends_at` timestamp DEFAULT null,
  `timezone` varchar(255) NOT NULL DEFAULT 'Europe/London',
  `currency` varchar(255) NOT NULL DEFAULT 'GBP',
  `disable_notifications` tin

In [ ]:
# Cell 7: Analyze input data
start_time = time.time()

print("Starting input analysis...")
analysis_prompt = f"""Analyze this JSON input and identify:
1. Missing required fields based on schema
2. Fields needing format conversion (especially dates and names)
3. Data quality issues or inconsistencies

Input JSON: {json.dumps(test_json, indent=2)}

Schema Context: {relevant_chunks[0].page_content}

Provide analysis in this format:
- Missing Fields: [list them]
- Format Conversions Needed: [field: required format]
- Quality Issues: [list any found]"""

analysis = llm.invoke(analysis_prompt)
print("\nInput Analysis Results:")
print(analysis)
print(f"\nAnalysis length: {len(analysis)} characters")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Starting input analysis...

Input Analysis Results:
1. Missing Fields:
   - `addressable_type` (should be the type of object this address belongs to, e.g., "Person" or "Business")
   - `created_at` (timestamp)

  2. Format Conversions Needed:
   - `fullAddress`: This field should ideally be broken down into separate fields like `address_line_1`, `address_line_2`, etc., according to the schema.
   - `postCode`: Should be in a consistent format that matches the schema's `postcode_zipcode`.
   - `gender`: The value "Male" does not match the expected gender values (either "Female" or "Male").
   - `telephone`: This field should contain a valid phone number in an appropriate format.
   - `contractNumber` and `regNumber`: These fields should contain numeric values.
   - `commissionFlatRate` and `commissionPercent`: These fields should be numerical values, not strings.
   - `timeSlot`: This field should be a numerical value representing the time slot.
   - `taxNo`: This field should contain a

In [ ]:
# Cell 8: Transform data
start_time = time.time()

print("Starting data transformation...")
transform_prompt = f"""Transform this JSON to match schema requirements:

Input JSON: {json.dumps(test_json, indent=2)}

Relevant Schema Context: {relevant_chunks[0].page_content}

Analysis Results: {analysis}

Requirements:
1. Follow schema data types exactly
2. Handle missing fields appropriately
3. Convert date formats to ISO (YYYY-MM-DD)
4. Split name fields if needed
5. Ensure all required fields are present

Return valid JSON only without any explanation."""

print("Sending transformation prompt...")
transformed_result = llm.invoke(transform_prompt)
print(f"Received response length: {len(transformed_result)} characters")

# Try to parse the result as JSON
try:
    transformed_data = json.loads(transformed_result)
    print("\nSuccessfully parsed transformed data:")
    print(json.dumps(transformed_data, indent=2))
    print(f"\nNumber of fields in output: {len(transformed_data.keys())}")
    print("Output fields:", list(transformed_data.keys()))
except json.JSONDecodeError as e:
    print("\nWarning: Output was not valid JSON")
    print("Error:", str(e))
    print("\nRaw output:")
    print(transformed_result)

print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Starting data transformation...
Sending transformation prompt...
Received response length: 1279 characters

Error: Expecting property name enclosed in double quotes: line 18 column 27 (char 546)

Raw output:
 {
      "id": 678669,
      "addressable_type": "Person",
      "addressable_id": 678669,
      "address_line_1": "125 Jackson Ranch",
      "address_line_2": "",
      "address_line_3": "",
      "address_line_4": "",
      "address_line_5": "",
      "town_city": "Port Damienberg",
      "state_county": "Lancashire",
      "postcode_zipcode": "TS24 7QA",
      "country": "Scotland",
      "created_at": "YYYY-MM-DD HH:mm:ss",
      "fullName": "Lizzie Brown",
      "forename": "Lizzie",
      "surname": "Brown",
      "gender": "Female", // Corrected based on fullName field
      "locationName": 771115, // Assuming locationName is the id of the location
      "title": "Miss",
      "telephone": "+44-07608-506942", // Formatted phone number
      "email": "uhughes@example.net",
  

In [ ]:
start_time = time.time()

print("Searching for relevant schema parts...")
results = vector_store.similarity_search_with_score(
    json.dumps(test_json),
    k=10,
    filter={"type": "chunk"}
)

relevant_chunks = [doc for doc, score in results if score < 0.8]

print(f"\nFound {len(relevant_chunks)} relevant chunks")
print("\nMost relevant chunks with scores:")
for i, (doc, score) in enumerate(results, 1):
    print(f"\nChunk {i}:")
    if 'CREATE TABLE' in doc.page_content:
        table_name = doc.page_content.split('`')[1]
        print(f"Table: {table_name}")
        print(f"Similarity Score: {score:.4f}")
    print(doc.page_content)
    print(f"Length: {len(doc.page_content)} characters")

print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Searching for relevant schema parts...

Found 0 relevant chunks

Most relevant chunks with scores:

Chunk 1:
Table: addresses
Similarity Score: 1.1762
CREATE TABLE `addresses` (
  `id` bigint PRIMARY KEY NOT NULL,
  `addressable_type` varchar(255) NOT NULL,
  `addressable_id` bigint NOT NULL,
  `address_line_1` varchar(255) DEFAULT null,
  `address_line_2` varchar(255) DEFAULT null,
  `address_line_3` varchar(255) DEFAULT null,
  `town_city` varchar(255) DEFAULT null,
  `state_county` varchar(255) DEFAULT null,
  `postcode_zipcode` varchar(255) DEFAULT null,
  `country` varchar(255) DEFAULT null,
  `created_at` timestamp DEFAULT null,
Length: 491 characters

Chunk 2:
`stripe_id` varchar(255) DEFAULT null,
  `rehabmypatient_api_key` text,
  `mailerlite_api_key` text,
  `pm_type` varchar(255) DEFAULT null,
  `pm_last_four` varchar(4) DEFAULT null,
  `trial_ends_at` timestamp DEFAULT null,
  `timezone` varchar(255) NOT NULL DEFAULT 'Europe/London',
  `currency` varchar(255) NOT NULL DEFAU

In [ ]:
start_time = time.time()

print("Starting input analysis...")
analysis_prompt = f"""CHECK THESE STEPS EXACTLY:

1. Find a CREATE TABLE statement in the schema that starts with "CREATE TABLE `"
2. Only proceed if you find an exact match - do not invent or modify tables
3. Copy that exact CREATE TABLE statement as proof
4. Match input JSON fields to the actual columns in that CREATE TABLE statement

Input JSON: {json.dumps(test_json, indent=2)}

Available Schema:
{chr(10).join(f"Table {i+1}: {chunk.page_content}" for i, chunk in enumerate(relevant_chunks))}

Steps required in your response:
1. Quote the exact CREATE TABLE statement you found (must exist in schema)
2. Show which input fields match to which actual table columns
3. List which schema columns need NULL/defaults
4. If you cannot find a real CREATE TABLE statement, say "No matching table found"
"""

analysis = llm.invoke(analysis_prompt)
print("\nAnalysis Results:")
print(analysis)
print(f"\nAnalysis length: {len(analysis)} characters")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Starting input analysis...

Analysis Results:
1. Quote the exact CREATE TABLE statement I found (must exist in schema):

I'm unable to provide an exact `CREATE TABLE` statement as the provided schema does not contain any table that starts with "CREATE TABLE `". It seems like you might have missed or omitted the relevant table creation statements.

  2. Show which input fields match to which actual table columns:

Here is a suggested mapping between the input JSON fields and potential table columns based on the schema provided:

- id (input) -> id (schema, if it exists)
- fullName (input) -> name (schema, assuming there's a 'name' column in the table)
- name (input) -> name (schema, assuming there's a 'name' column in the table)
- code (input) -> code (schema, if it exists)
- fullAddress (input) -> address (schema, assuming there's an 'address' column in the table)
- locationName (input) -> location_name (schema, assuming there's a 'location_name' column in the table)
- title (input) ->

In [ ]:
start_time = time.time()

print("Starting input analysis...")
analysis_prompt = f"""You are an AI Data engineer and you are required to match input json to your company's schema. It is extremely important to not infer a table or table name based on input json data, only use the exact names/titles of the tables present in schema. Do not add any fields/column in input json that are not in the schema into the output, the goal is tho match the schema exactly to enable easy data entry and onboarding.
For all future outputs, only use the exact names/titles of the fields present in schema, if a field in input is not present in schema, forget about it as we want to match the schema exactly to enable easy data entry and onboarding. 

Analyze this JSON input against ALL relevant schema tables and prove that the name of the table from the schema was named as is, by mentioning the position of the table in the schema to back up your claim:

Input JSON: {json.dumps(test_json, indent=2)}

Schema Tables:
{chr(10).join(f"Table {i+1}: {chunk.page_content}" for i, chunk in enumerate(relevant_chunks))}



DO:
identify all the tables present in the relevant schema chunk that are relevant to the input json for transformation. It is extremely important to not infer a table or table name based on input json data, only use the exact names/titles of the tables present in schema

state the proof that the name of the table from the schema was named as is, by mentioning the position of the table in the schema to back up your claim, state the line number of said table in schema


"""

analysis = llm.invoke(analysis_prompt)
print("\nAnalysis Results:")
print(analysis)
print(f"\nAnalysis length: {len(analysis)} characters")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Starting input analysis...

Analysis Results:
 Based on the provided Input JSON and Schema Tables, I can identify three tables relevant to the data: "Employee", "Location", and "Service".

   Here's the proof that the names of these tables from the schema were named as is, by mentioning the line number of said tables in schema:

   - The "Employee" table is present on line 10 of the Schema Tables. The JSON data contains fields such as "id", "fullName", "name", "code", "fullAddress", "locationName", "title", "forename", "surname", "gender", "addressLine1", "addressLine2", "addressLine3", "addressLine4", "addressLine5", "postCode", "telephone", "mobile", "email", "fax", "web", "online", "status", and "employmentStatus". These fields match the column names in the "Employee" table of the schema.

   - The "Location" table is present on line 15 of the Schema Tables. The JSON data contains fields such as "locationName", "addressLine1", "addressLine2", "addressLine3", "addressLine4", "address

In [ ]:
start_time = time.time()

print("Starting data transformation...")
transform_prompt = f"""Transform this JSON to match schema requirements:

Input JSON: {json.dumps(test_json, indent=2)}

Schema Requirements:
{chr(10).join(f"Table {i+1}: {chunk.page_content}" for i, chunk in enumerate(relevant_chunks))}

Analysis Results: {analysis}

Return JSON with field names of the schema as keys and transformed data as values. The keys should match exactly to the column/field names in the schema and any field given in input json that is not a field in the schema shoud be dropped. All fields in the schema should be present and missing fields should be filled with null. Do not add any new fields or tables. Remove any field or columns in the input json that is not in the schema especially fields about names."""

print("Sending transformation prompt...")
transformed_result = llm.invoke(transform_prompt)
print(f"Received response length: {len(transformed_result)} characters")

try:
    transformed_data = json.loads(transformed_result)
    print("\nTransformed data:")
    print(json.dumps(transformed_data, indent=2))
    print(f"\nTables transformed: {list(transformed_data.keys())}")
except json.JSONDecodeError as e:
    print("\nWarning: Invalid JSON output")
    print(f"Error: {str(e)}")
    print("\nRaw output:")
    print(transformed_result)

print(f"\nTime taken: {time.time() - start_time:.2f} seconds")

Starting data transformation...
Sending transformation prompt...
Received response length: 1601 characters

Error: Expecting value: line 1 column 2 (char 1)

Raw output:
 Here's the transformed JSON based on the schema requirements:

```JSON
{
  "EmployeeId": null,
  "FirstName": "Lizzie",
  "LastName": "Brown",
  "Gender": "Female", // Corrected from "Male" to match the title "Miss"
  "LocationId": 771115,
  "ServiceProviderId": null,
  "ContactId": null,
  "Address_Line1": "125 Jackson Ranch",
  "Address_Line2": "",
  "Address_Line3": "",
  "Address_Line4": "",
  "Address_Line5": "",
  "PostCode": "TS24 7QA",
  "Telephone": "", // This field might not be acceptable depending on the business rules
  "Mobile": "07608 506942",
  "Email": "uhughes@example.net",
  "Fax": "",
  "Web": "",
  "Online": true,
  "Status": 1,
  "Type": "Acupuncturist",
  "ContractNumber": "", // This field might not be acceptable depending on the business rules
  "RegNumber": "", // This field might not be acce